# Mixing bayesian methods
Evan Edwards

## Multivariate + sigmoid bivariate

In [3]:
from Taweret.core.base_model import BaseModel
import matplotlib.pyplot as plt
import numpy as np
import pymc as pm
import pandas as pd
from sklearn.metrics import r2_score, mean_squared_error
import bilby
import ptemcee

In [4]:
# Load dataset
PISA2018 = pd.read_csv("pisa2018.BayesBook.csv")
# Data processing: converting categorical values to numerical values
PISA2018['Female'] = PISA2018['Female'].replace({'Female': 1.0, 'Male': 0.0})
# Converting numerical to categorical values
PISA2018['SchoolID'] = pd.Categorical(PISA2018['SchoolID']).codes

PISA2018 = PISA2018[["PV1READ","Female","ESCS", "HOMEPOS", "ICTRES", "JOYREAD", "WORKMAST","PISADIFF","TEACHINT","BELONG","SCREADCOMP","SCREADDIFF","METASUM","GFOFAIL","MASTGOAL","SWBP","ADAPTIVITY","SchoolID","COMPETE","PERFEED"]]

In [5]:
#PV1READ ~ Female + ESCS + HOMEPOS + ICTRES + (1 + ICTRES | SchoolID)
with pm.Model() as model1:
    Female = pm.MutableData("Female",PISA2018["Female"])
    ESCS = pm.MutableData("ESCS", PISA2018["ESCS"])
    HOMEPOS = pm.MutableData("HOMEPOS", PISA2018["HOMEPOS"])
    ICTRES = pm.MutableData("ICTRES", PISA2018["ICTRES"])
    SchoolID = pm.MutableData("SchoolID", PISA2018["SchoolID"])

    alpha = pm.Normal('alpha', mu=0, sigma=100)  # Intercept
    beta_female = pm.Uniform('beta_female', lower=0, upper=1)  # Coefficient for Female
    beta_escs = pm.Normal('beta_escs', mu=np.mean(PISA2018["ESCS"]), sigma = np.std(PISA2018["ESCS"]))  # Coefficient for ESCS
    beta_homepos = pm.StudentT('beta_homepos', nu = 1,mu=np.mean(PISA2018["HOMEPOS"]), sigma=100)  # Coefficient for HOMEPOS
    beta_ictres = pm.Normal('beta_ictres', mu=np.mean(PISA2018["ICTRES"]), sigma=np.std(PISA2018["ICTRES"]))  # Coefficient for ICTRES

    sigma_school = pm.HalfNormal('sigma_school', sigma=100)
    sigma_ictres = pm.HalfNormal('sigma_ictres', sigma=100)
    intercept_school = pm.Normal('intercept_school', mu=0, sigma=sigma_school, shape=len(PISA2018["SchoolID"].unique()))
    random_ictres = pm.Normal('random_ictres', mu=0, sigma=sigma_ictres, shape=len(PISA2018["SchoolID"].unique()))

    PV1READ_mu = alpha + beta_female * Female + beta_escs * ESCS + beta_homepos * HOMEPOS\
        + beta_ictres * ICTRES + random_ictres[SchoolID] * ICTRES  + intercept_school[SchoolID]
    PV1READ_sigma = pm.HalfNormal('PV1READ_sigma', sigma=100)

    PV1READ = pm.Normal('PV1READ', mu=PV1READ_mu, sigma=PV1READ_sigma, observed=PISA2018['PV1READ'])
    

with model1:
    trace1 = pm.sample(1000, tune=2000)

    
with model1:
    post_pred = pm.sample_posterior_predictive(trace1, model=model1)
pred = post_pred.posterior_predictive["PV1READ"].mean(dim=["chain", "draw"])


print(f'The R^2 score is: {r2_score(PISA2018["PV1READ"], pred)}')
print(f'The MSE is: {mean_squared_error(PISA2018["PV1READ"], pred)}')


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta_female, beta_escs, beta_homepos, beta_ictres, sigma_school, sigma_ictres, intercept_school, random_ictres, PV1READ_sigma]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 131 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Sampling: [PV1READ]


The R^2 score is: 0.24416136084689133
The MSE is: 8888.683339577454


In [6]:
#PV1READ ~ JOYREAD + PISADIFF + SCREADCOMP + SCREADDIFF + (1|SchoolID)
with pm.Model() as model2:
    JOYREAD = pm.MutableData("JOYREAD",PISA2018["JOYREAD"])
    PISADIFF = pm.MutableData("PISADIFF", PISA2018["PISADIFF"])
    SCREADCOMP = pm.MutableData("SCREADCOMP", PISA2018["SCREADCOMP"])
    SCREADDIFF = pm.MutableData("SCREADDIFF", PISA2018["SCREADDIFF"])
    SchoolID = pm.MutableData("SchoolID", PISA2018["SchoolID"])

    alpha = pm.Normal('alpha', mu=0, sigma=100)
    beta_joyread = pm.Normal('beta_joyread', mu=np.mean(PISA2018["JOYREAD"]), sigma=np.std(PISA2018["JOYREAD"]))  # Coefficient for JOYREAD
    beta_pisadiff = pm.Normal('beta_pisadiff', mu=0, sigma=100)  # Coefficient for PISADIFF
    beta_screadcomp = pm.Normal('beta_screadcomp', mu=np.mean(PISA2018["SCREADCOMP"]), sigma=10)  # Coefficient for SCREADCOMP
    beta_screaddiff = pm.Normal('beta_screaddiff', mu=np.mean(PISA2018["SCREADDIFF"]), sigma=np.std(PISA2018["SCREADDIFF"]))  # Coefficient for SCREADDIFF

    sigma_school = pm.HalfNormal('sigma_school', sigma=100)
    intercept_school = pm.Normal('intercept_school', mu=0, sigma=sigma_school, shape=len(PISA2018["SchoolID"].unique()))

    PV1READ_mu = alpha + beta_joyread * JOYREAD + beta_pisadiff * PISADIFF + beta_screadcomp * SCREADCOMP\
        + beta_screaddiff * SCREADDIFF  + intercept_school[SchoolID]
    PV1READ_sigma = pm.HalfNormal('PV1READ_sigma', sigma=100)
    PV1READ = pm.Normal('PV1READ', mu=PV1READ_mu, sigma=PV1READ_sigma, observed=PISA2018['PV1READ'])
    

with model2:
    trace2 = pm.sample(1000, tune=2000)


with model2:
    post_pred = pm.sample_posterior_predictive(trace2, model=model2)
pred = post_pred.posterior_predictive["PV1READ"].mean(dim=["chain", "draw"])

print(f'The R^2 score is: {r2_score(PISA2018["PV1READ"], pred)}')
print(f'The MSE is: {mean_squared_error(PISA2018["PV1READ"], pred)}')

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta_joyread, beta_pisadiff, beta_screadcomp, beta_screaddiff, sigma_school, intercept_school, PV1READ_sigma]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 88 seconds.
Sampling: [PV1READ]


The R^2 score is: 0.3700315992716384
The MSE is: 7408.445848029931


In [7]:
#PV1READ ~ METASUM + GFOFAIL + MASTGOAL + SWBP + WORKMAST + ADAPTIVITY + COMPETE + (1|SchoolID) 
with pm.Model() as model3:
    METASUM = pm.MutableData("METASUM",PISA2018["METASUM"])
    GFOFAIL = pm.MutableData("GFOFAIL", PISA2018["GFOFAIL"])
    MASTGOAL = pm.MutableData("MASTGOAL", PISA2018["MASTGOAL"])
    SWBP = pm.MutableData("SWBP", PISA2018["SWBP"])
    WORKMAST = pm.MutableData("WORKMAST", PISA2018["WORKMAST"])
    ADAPTIVITY = pm.MutableData("ADAPTIVITY", PISA2018["ADAPTIVITY"])
    COMPETE = pm.MutableData("COMPETE", PISA2018["COMPETE"])
    SchoolID = pm.MutableData("SchoolID", PISA2018["SchoolID"])

    alpha = pm.Normal('alpha', mu=0, sigma=100)  # Intercept
    beta_metasum = pm.Normal('beta_metasum', mu=np.mean(PISA2018["METASUM"]), sigma=np.std(PISA2018["METASUM"]))  # Coefficient for METASUM
    beta_gfofail = pm.Normal('beta_gfofail', mu=0, sigma=100)  # Coefficient for GFOFAIL
    beta_mastgoal = pm.Normal('beta_mastgoal', mu=np.mean(PISA2018["MASTGOAL"]), sigma=10)  # Coefficient for MASTGOAL
    beta_swbp = pm.Normal('beta_swbp', mu=0, sigma=100)  # Coefficient for SWBP
    beta_workmast = pm.Normal('beta_workmast', mu=np.mean(PISA2018["WORKMAST"]), sigma=10)  # Coefficient for WORKMAST
    beta_adaptivity = pm.Normal('beta_adaptivity', mu=np.mean(PISA2018["ADAPTIVITY"]), sigma=100)  # Coefficient for ADAPTIVITY
    beta_compete = pm.Normal('beta_compete', mu=np.mean(PISA2018["COMPETE"]), sigma=np.std(PISA2018["COMPETE"]))  # Coefficient for COMPETE

    sigma_school = pm.HalfNormal('sigma_school', sigma=100)
    intercept_school = pm.StudentT('intercept_school', nu = 1, mu=0, sigma=sigma_school, shape=len(PISA2018["SchoolID"].unique()))

    PV1READ_mu = alpha + beta_metasum * METASUM + beta_gfofail * GFOFAIL + beta_mastgoal * MASTGOAL\
        + beta_swbp * SWBP + beta_workmast * WORKMAST + beta_adaptivity * ADAPTIVITY\
        + beta_compete * COMPETE  + intercept_school[SchoolID]
    PV1READ_sigma = pm.HalfStudentT('PV1READ_sigma', nu = 10, sigma=100)
    PV1READ = pm.Normal('PV1READ', mu=PV1READ_mu, sigma=PV1READ_sigma, observed=PISA2018['PV1READ'])
    

with model3:
    trace3 = pm.sample(1000, tune=2000)

with model3:
    post_pred = pm.sample_posterior_predictive(trace3, model=model3)
pred = post_pred.posterior_predictive["PV1READ"].mean(dim=["chain", "draw"])

# Printing the accuracy metrics
print(f'The R^2 score is: {r2_score(PISA2018["PV1READ"], pred)}')
print(f'The MSE is: {mean_squared_error(PISA2018["PV1READ"], pred)}')

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta_metasum, beta_gfofail, beta_mastgoal, beta_swbp, beta_workmast, beta_adaptivity, beta_compete, sigma_school, intercept_school, PV1READ_sigma]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 2267 seconds.
The rhat statistic is larger than 1.01 for some parameters. This indicates problems during sampling. See https://arxiv.org/abs/1903.08008 for details
The effective sample size per chain is smaller than 100 for some parameters.  A higher number is needed for reliable rhat and ess computation. See https://arxiv.org/abs/1903.08008 for details
Chain 0 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 1 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Chain 2 reached the maximum tree depth. Increase `max_treedepth`, increase `target_accept` or reparameterize.
Sampling: [PV1READ]


The R^2 score is: 0.20832643123731398
The MSE is: 9310.103104717376


In [8]:
#PV1READ ~ PERFEED + TEACHINT + BELONG + (1 + TEACHINT | SchoolID)
with pm.Model() as model4:
    PERFEED = pm.MutableData("PERFEED",PISA2018["PERFEED"])
    TEACHINT = pm.MutableData("TEACHINT", PISA2018["TEACHINT"])
    BELONG = pm.MutableData("BELONG", PISA2018["BELONG"])
    SchoolID = pm.MutableData("SchoolID", PISA2018["SchoolID"])

    
    alpha = pm.Normal('alpha', mu=0, sigma=100)
    beta_perfeed = pm.Normal('beta_perfeed', mu = np.mean(PISA2018["PERFEED"]), sigma = np.std(PISA2018["PERFEED"]))  # Coefficient for PERFEED
    beta_teachint = pm.Normal('beta_teachint', mu=np.mean(PISA2018["TEACHINT"]), sigma = np.std(PISA2018["TEACHINT"]))  # Coefficient for TEACHINT
    beta_belong = pm.StudentT('beta_belong', nu = 1, mu=np.mean(PISA2018["BELONG"]), sigma=100)  # Coefficient for BELONG

    sigma_school = pm.HalfNormal('sigma_school', sigma=100)
    sigma_teachint = pm.HalfNormal('sigma_teachint', sigma=100)
    intercept_school = pm.Normal('intercept_school', mu=0, sigma=sigma_school, shape=len(PISA2018["SchoolID"].unique()))
    random_teachint = pm.Normal('random_teachint', mu=0, sigma=sigma_teachint, shape=len(PISA2018["SchoolID"].unique()))

    PV1READ_mu = alpha + beta_perfeed * PERFEED + beta_teachint * TEACHINT + beta_belong * BELONG\
        + random_teachint[SchoolID] * TEACHINT  + intercept_school[SchoolID]
    PV1READ_sigma = pm.HalfNormal('PV1READ_sigma', sigma=100)
    PV1READ = pm.Normal('PV1READ', mu=PV1READ_mu, sigma=PV1READ_sigma, observed=PISA2018['PV1READ'])
    

with model4:
    trace4 = pm.sample(1000, tune=2000)

with model4:
    post_pred = pm.sample_posterior_predictive(trace4)
pred = post_pred.posterior_predictive["PV1READ"].mean(dim=["chain", "draw"])

# Printing the accuracy metrics
print(f'The R^2 score is: {r2_score(PISA2018["PV1READ"], pred)}')
print(f'The MSE is: {mean_squared_error(PISA2018["PV1READ"], pred)}')

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, beta_perfeed, beta_teachint, beta_belong, sigma_school, sigma_teachint, intercept_school, random_teachint, PV1READ_sigma]


Sampling 4 chains for 2_000 tune and 1_000 draw iterations (8_000 + 4_000 draws total) took 145 seconds.
Sampling: [PV1READ]


The R^2 score is: 0.23627606471682405
The MSE is: 8981.414640556584


In [9]:
class PYMCWrapper(BaseModel):
    prior = []
    def __init__(self, model, trace, pred_value):
        self.pymcmodel = model
        self.trace = trace
        self.pred_value = pred_value

    def evaluate(self, model_parameters):
        with self.pymcmodel:
            for entry in model_parameters:
                try:
                    pm.set_data(entry)
                except:
                    pass
            post_pred_new = pm.sample_posterior_predictive(self.trace)

        return np.array(post_pred_new.posterior_predictive[self.pred_value].mean(dim=["chain", "draw"]).data),\
                np.array(post_pred.posterior_predictive["PV1READ"].var(dim=["chain", "draw"]))

    
    def log_likelihood_elementwise(self,x_exp, y_exp, y_err, model_params):
        # Assuming a normal distribution for error
        y = self.evaluate(model_params)[0]
        # If y_exp, y_err, y are numpy arrays of same length
        return np.exp(-(y - y_exp) **2 / (2 * y_err ** 2)) \
            / np.sqrt(2 * np.pi * y_err ** 2)
    
    def set_prior(self, prior_dict):
        with self.pymcmodel:
            for key, value in prior_dict.items():
                self.pymcmodel.key = value


In [10]:
m1 = PYMCWrapper(model1, trace1, "PV1READ")
m2 = PYMCWrapper(model2, trace2, "PV1READ")
m3 = PYMCWrapper(model3, trace3, "PV1READ")
m4 = PYMCWrapper(model4, trace4, "PV1READ")

In [14]:
from Taweret.mix.gaussian import Multivariate

In [15]:
models = {
    "1": m1,
    "2": m2,
    "3": m3,
    "4": m4
}

print(models, type(models))

{'1': <__main__.PYMCWrapper object at 0x000001D608291250>, '2': <__main__.PYMCWrapper object at 0x000001D608E00CD0>, '3': <__main__.PYMCWrapper object at 0x000001D606F6F390>, '4': <__main__.PYMCWrapper object at 0x000001D606F6FD50>} <class 'dict'>


In [16]:

#Defining the mixed model
mixed = Multivariate(PISA2018, models)
draws, mean, intervals, std_dev = mixed.predict(ci=68)

Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


In [18]:
print(f'The R^2 score is: {r2_score(PISA2018["PV1READ"], mean)}')
print(f'The MSE is: {mean_squared_error(PISA2018["PV1READ"], mean)}')

The R^2 score is: 0.32481592813115645
The MSE is: 7940.183393499344


In [19]:
from Taweret.mix.bivariate_linear import BivariateLinear as BL

In [78]:
#Training the bivariate mixer
bi_models = {
    "1": m1,
    "2": m2
}

print(bi_models, type(bi_models))
variate_mixer_sig = BL(models_dic=bi_models, method='sigmoid', same_parameters=False)

{'1': <__main__.PYMCWrapper object at 0x000001D608291250>, '2': <__main__.PYMCWrapper object at 0x000001D608E00CD0>} <class 'dict'>
sigmoid mixing function has 2                   free parameter(s)
To change the prior use `set_prior` method


In [71]:
# Define priors for mixing function parameters
mixing_priors = {'sigmoid_0': bilby.core.prior.Normal(0, 1, name='sigmoid_0'),
                 'sigmoid_1': bilby.core.prior.Normal(0, 1, name='sigmoid_1')}

# Set priors for model parameters
model_priors = {'model1_0': bilby.core.prior.Normal(0,1),
                'model1_1': bilby.core.prior.Normal(0,1),
                'model2_0': bilby.core.prior.Normal(0,1),
                'model2_1': bilby.core.prior.Normal(0,1)}

# Combine mixing function priors and model priors
all_priors = {**mixing_priors, **model_priors}

# Set priors in the bivariate_mixer
variate_mixer_sig.set_prior(all_priors)

{'step_0': Uniform(minimum=0, maximum=1, name='step_0', latex_label='step_0', unit=None, boundary=None),
 'step_1': Uniform(minimum=0, maximum=1, name='step_1', latex_label='step_1', unit=None, boundary=None),
 'model1_0': Uniform(minimum=0, maximum=1, name=None, latex_label=None, unit=None, boundary=None),
 'model1_1': Uniform(minimum=0, maximum=1, name=None, latex_label=None, unit=None, boundary=None),
 'model2_0': Uniform(minimum=0, maximum=1, name=None, latex_label=None, unit=None, boundary=None),
 'model2_1': Uniform(minimum=0, maximum=1, name=None, latex_label=None, unit=None, boundary=None)}

In [65]:
#Training the bivariate mixer
kwargs_for_sampler = {'sampler': 'Emcee','nwalkers':200,'nsteps':100}

x_exp = np.array(PISA2018).reshape(20,4838)
y_exp = np.array(PISA2018["PV1READ"]).reshape(1,-1)
y_err = np.array(PISA2018["PV1READ"]).reshape(1,-1)-np.mean(PISA2018["PV1READ"]).reshape(1,-1)
result = variate_mixer_sig.train(x_exp, y_exp, y_err, outdir='outdir\pymc_variate_mixer_sig', label='variate_mixer_sig',kwargs_for_sampler=kwargs_for_sampler);

In [66]:
# Access the posterior samples
posterior_samples = variate_mixer_sig.posterior

predictions, mean, credible_intervals, std_dev = variate_mixer_sig.predict(x_exp)

Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


...

Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


Sampling: [PV1READ]


In [63]:
#Accuracy of bivariate
pred_bv=np.mean(np.array(predictions),axis=(0,1))
# Printing the accuracy metrics
print(f'The R^2 score is: {r2_score(PISA2018["PV1READ"], pred_bv)}')
print(f'The MSE is: {mean_squared_error(PISA2018["PV1READ"], pred_bv)}')

The R^2 score is: 0.34076850782093215
The MSE is: 7752.580614325908
